# Example: Use RegioSQM2018

Reference
- https://github.com/jensengroup/regiosqm
- https://pubs.rsc.org/en/content/articlelanding/2018/SC/C7SC04156J


In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import logging
import sys

In [ ]:
# Show progress bars on pandas functions
from tqdm.auto import tqdm
tqdm.pandas()

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import SVG
from rdkit import Chem
from rdkit.Chem import AllChem, PandasTools
from rdkit.Chem.Draw import MolsToGridImage, MolToImage, rdMolDraw2D

In [ ]:
import ppqm

In [ ]:
# Set logging
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger("xtb").setLevel(logging.INFO)
show_progress = True

In [ ]:
# Set DataFrames visuals
PandasTools.RenderImagesInAllDataFrames(images=True)
pd.set_option('display.float_format','{:.2f}'.format)
from IPython.display import HTML

## Import regiosqm

In [ ]:
import regiosqm_lib as regiolib
from regiosqm_lib.methods import regiosqm2018

# Define molecule





In [ ]:
#smiles = "Cc1cc(NCCO)nc(-c2ccc(Br)cc2)n1"  # CHEMBL1956589
#smiles = "n1(C)ccnc1"
#smiles = "c1cnc(N)c(O[C@@H](c2cc(Cl)ccc2C(F)(F)F)C)c1"
smiles = "c1(N(C)C)cccnc1"
smiles = "c1(c(ccc(c1)N)F)[C@]1(NC(N(S(=O)(=O)C1)C)NC(=O)OC(C)(C)C)C"
# smiles = "n1cccn1c1ncccn1"
molobj = Chem.MolFromSmiles(smiles)

In [ ]:
mol_ = Chem.Mol(molobj, True)
atoms = mol_.GetNumAtoms()
for idx in range( atoms ):
    mol_.GetAtomWithIdx( idx ).SetProp( 'molAtomMapNumber', str( mol_.GetAtomWithIdx( idx ).GetIdx() ) )
HTML(PandasTools.PrintAsBase64PNGString(mol_))

# Generate and calculate energies of tautomers and protonations

In [ ]:
%%time
pdf = regiosqm2018.predict_regioselective_dataframe(molobj)

# Overview of all target sites and energies

In [ ]:
HTML(pdf.to_html())

# With the all energies, select green and red sites

In [ ]:
mol = regiosqm2018.predict_regioselective_sites(molobj, pdf)

In [ ]:
mol

In [ ]:
green_indices = mol.GetProp("regiosqm2018_cut1").strip('][').split(', ')
red_indices = mol.GetProp("regiosqm2018_cut2").strip('][').split(', ')

## Show results

In [ ]:
# Define pretty colors
colors = dict()
colors["green"] = (119, 198, 110)
colors["green"] = tuple(x/255 for x in colors["green"])
colors["red"] = (201, 43, 38)
colors["red"] = tuple(x/255 for x in colors["red"])

# Find reactive centers and convert index type to int.
# rdkit doesn't understand np.int
green_indices = [int(x) for x in green_indices if x]
red_indices = [int(x) for x in red_indices if x not in green_indices and x]

# All highlights
highlights = green_indices + red_indices

# Map highlight to a color
colormap = dict()
colormap.update({key: [colors["green"]] for key in green_indices})
colormap.update({key: [colors["red"]] for key in red_indices})

In [ ]:
# should be working, but does not respect colors
# MolToImage(
#    molobj,
#    highlightAtoms=highlights,
#    highlightMap=colormap,
#    size=(500,500),
#)

In [ ]:
# http://rdkit.blogspot.com/2020/04/new-drawing-options-in-202003-release.html
d2d = rdMolDraw2D.MolDraw2DSVG(500, 500)
d2d.DrawMoleculeWithHighlights(molobj, "Regioselective site(s)", dict(colormap), {}, {}, {})
d2d.FinishDrawing()
SVG(d2d.GetDrawingText())